# Image recognition with Python, OpenCV

## Step 1: Install requirements

In [1]:
!pip install opencv-python imgbeddings psycopg2-binary

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
  Created wheel for imgbeddings: filename=imgbeddings-0.1.0-py3-none-any.whl size=9324 sha256=fe9c0906437ea20b27a93dc23cab4e8df00c6f41de29f647afbf25986e4873f2
  Stored in directory: /root/.cache/pip/wheels/44/25/45/faf940ae980b321ddb51becf6a38c32800af42853801c17cd0
Successfully built imgbeddings


In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1: Face recognition

Detect the faces from the [test-image](test-image.png) picture and store them under the `stored-faces` folder

In [12]:
# importing the cv2 library, os and the others needed
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt


# loading the haar case algorithm file into alg variable
alg = "/content/drive/MyDrive/Colab Notebooks/Face_recognition/haarcascade_frontalface_default.xml"

# passing the algorithm to OpenCV
haar_cascade = cv2.CascadeClassifier(alg)

# creating the extract_face function
def extract_face(filename, required_size=(160, 160)):

    #Extracts face from an image using HAAR cascade
    img = cv2.imread(filename)
    if img is None:
        return None
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to gray scale image
    faces = haar_cascade.detectMultiScale(gray_img, 1.1, 4) # 1.1 is the scale-factor while 4 is MinNeighbor

    if len(faces) == 0:
        return None  # No face found

    x, y, w, h = faces[0]  # Take the first detected face
    face = img[y:y+h, x:x+w]  # Crop the face
    face = cv2.resize(face, required_size)  # Resize to (160,160)
    return face

In [9]:
def load_faces(directory):

    # Loads all the faces from the directory
    faces = []
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)
        face = extract_face(path)
        if face is not None:
            faces.append(face)
    return faces

In [10]:
def load_dataset(parent_directory):

    # Loading the dataset and assigning labels based on folder names
    X, y = [], []

    for subdir in os.listdir(parent_directory):
        path = os.path.join(parent_directory, subdir)
        if not os.path.isdir(path):
            continue  # Skip files
        faces = load_faces(path)
        labels = [subdir] * len(faces)  # Assign label to each face
        X.extend(faces)
        y.extend(labels)

    return np.array(X), np.array(y)

In [13]:
trainX, trainy = load_dataset('/content/drive/MyDrive/Colab Notebooks/Face_recognition/dataset/Train')
testX, testy = load_dataset('/content/drive/MyDrive/Colab Notebooks/Face_recognition/dataset/Test')

print(f"Train set: {trainX.shape}, {trainy.shape}")
print(f"Test set: {testX.shape}, {testy.shape}")

Train set: (769, 160, 160, 3), (769,)
Test set: (189, 160, 160, 3), (189,)


# Step 2: Face Embedding with FaceNet
Calculate embeddings from the faces

In [14]:
!pip install keras-facenet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=8e76e3082c13d98897a1315f6a0965e58adb14dda5cb55bacfca3e68c33f9906
  Stored in directory: /root/.cache/pip/wheels/99/94/dd/cb1a65a7440ba6d508bd24346c15af0b1d24ff8b1cdb1c9959
Successfully built keras-facenet


In [15]:
from keras_facenet import FaceNet
import numpy as np

In [17]:
# Load the FaceNet model
facenet_model = FaceNet()

# creating the get_embedding function
def get_embedding(model, face_image):
    """Generates an embedding for a given face using a pre-trained model."""
    face_image = np.expand_dims(face_image, axis=0)  # Add batch dimension
    embedding = model.embeddings(face_image)  # Get the face embedding
    return embedding[0]  # Return the 128-dimensional feature vector

In [18]:
# Generating embeddings for training images
train_embeddings = np.array([get_embedding(facenet_model, face) for face in trainX])
test_embeddings = np.array([get_embedding(facenet_model, face) for face in testX])

# Converting the labels (strings) into NumPy arrays
train_labels = np.array(trainy)
test_labels = np.array(testy)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [19]:
# Save train and test embeddings to a file
np.savez_compressed('/content/drive/MyDrive/Colab Notebooks/Face_recognition/face_embeddings.npz',
                    trainX=train_embeddings, trainy=train_labels,
                    testX=test_embeddings, testy=test_labels)